# 第五章：函式

## 學習目標
- 了解函式的宣告與呼叫
- 掌握具名回傳值的使用
- 學會處理可變參數
- 理解多個回傳值的機制
- 掌握函式作為值的概念
- 學會函式型態宣告
- 理解匿名函式和 closure
- 掌握高階函式的使用
- 學會 defer 語句的使用
- 理解 Go 的值呼叫機制

## 1. 宣告與呼叫函式

函式是 Go 程式的基本組建單位。函式將程式碼組織成可重複使用的區塊，使程式更加模組化和可維護。

### 函式的基本語法
```go
func 函式名稱(參數列表) 回傳型態 {
    函式主體
    return 回傳值
}
```

### 函式的組成部分
- **func 關鍵字**: 宣告函式的關鍵字
- **函式名稱**: 函式的識別符，遵循 Go 的命名規則
- **參數列表**: 函式接收的輸入，可以為空
- **回傳型態**: 函式回傳值的型態，可以省略（無回傳值）
- **函式主體**: 函式的實際程式碼
- **return 語句**: 回傳值給呼叫者，可選

### 重要概念
- 函式名稱首字母大寫表示公開（可匯出）
- 函式名稱首字母小寫表示私有（套件內使用）
- 參數是以值傳遞的方式傳入函式
- 函式可以有多個參數和多個回傳值

In [ ]:
package main

import "fmt"

// 簡單的函式宣告和呼叫
func greet() {
    fmt.Println("Hello, World!")
}

// 有參數的函式
func greetUser(name string) {
    fmt.Printf("Hello, %s!\n", name)
}

// 有參數和回傳值的函式
func add(a, b int) int {
    return a + b
}

// 多個參數，不同型態
func introduce(name string, age int, height float64) {
    fmt.Printf("姓名: %s, 年齡: %d, 身高: %.1fcm\n", name, age, height)
}

func main() {
    // 呼叫無參數函式
    greet()
    
    // 呼叫有參數的函式
    greetUser("Alice")
    
    // 呼叫有回傳值的函式
    result := add(5, 3)
    fmt.Printf("5 + 3 = %d\n", result)
    
    // 多參數函式
    introduce("Charlie", 25, 175.5)
}

## 2. 具名回傳值

Go 允許為回傳值命名，這些具名回傳值會在函式開始時被自動初始化為其型態的零值。

### 具名回傳值的特性
- **自動初始化**: 具名回傳值會被自動初始化為零值
- **提高可讀性**: 回傳值的名稱可以作為文件說明用途
- **裸回傳**: 可以使用空的 `return` 語句回傳所有具名回傳值. 
Caution, 用空回傳來回傳named return value是很差勁的design, 大家都要去找你上一個被assign的value是什麼. 
- **作為局部變數**: 具名回傳值可以在函式內被重新賦值

In [ ]:
package main

import (
    "fmt"
    "errors"
)

// 基本的具名回傳值
func divide(a, b float64) (result float64, err error) {
    if b == 0 {
        err = errors.New("除數不能為零")
        return // 裸回傳，回傳 0 和 error
    }
    result = a / b
    return // 裸回傳，回傳計算結果和 nil
}

// 計算圓的面積和周長
func circleInfo(radius float64) (area, circumference float64) {
    const pi = 3.14159
    area = pi * radius * radius
    circumference = 2 * pi * radius
    return // 裸回傳兩個計算結果
}

func main() {
    // 測試除法函式
    result1, err1 := divide(10, 2)
    if err1 != nil {
        fmt.Printf("錯誤: %v\n", err1)
    } else {
        fmt.Printf("10 / 2 = %.2f\n", result1)
    }
    
    // 測試圓的計算
    area, circumference := circleInfo(5.0)
    fmt.Printf("半徑 5 的圓 - 面積: %.2f, 周長: %.2f\n", area, circumference)
}

## 3. 參數

### 參數限制 (沒有named & optional parameter)

- Go 沒有 named & optional argument
- 你可以自己使用struct來模擬named parameter 


###  可變參數 (variadic) 與 slice

Go 支援可變參數函式，允許函式接受零個或多個相同型態的參數。可變參數使用 `...` 語法來定義，在函式內部表現為 slice。

#### 可變參數的特性
- **語法**: 使用 `...T` 表示型態 T 的可變參數
- **位置限制**: 可變參數必須是最後一個參數
- **內部表示**: 可變參數在函式內部是一個 slice
- **呼叫方式**: 兩種呼叫方式
    * 直接傳遞多個參數, 使用, 隔開
    * 使用 slice 加上 `...` 展開

In [ ]:
package main

import "fmt"

// 基本的可變參數函式：計算總和
func sum(numbers ...int) int {
    fmt.Printf("sum 函式接收到 %d 個參數: %v\n", len(numbers), numbers)
    total := 0
    for _, num := range numbers {
        total += num
    }
    return total
}

// 混合普通參數和可變參數
func formatMessage(prefix string, values ...interface{}) string {
    result := prefix
    for _, value := range values {
        result += fmt.Sprintf(" %v", value)
    }
    return result
}

func main() {
    // 基本用法：傳遞多個參數
    result1 := sum(1, 2, 3, 4, 5)
    fmt.Printf("sum(1,2,3,4,5) = %d\n", result1)
    
    // 沒有參數
    result2 := sum()
    fmt.Printf("sum() = %d\n", result2)
    
    // 使用 slice 展開
    numbers := []int{10, 20, 30, 40}
    result3 := sum(numbers...) // 注意 ... 語法
    fmt.Printf("sum(numbers...) = %d\n", result3)
    
    // 格式化訊息
    msg := formatMessage("Info:", "user", 123, true, 3.14)
    fmt.Printf("formatMessage: %s\n", msg)
}

## 4. 多個回傳值與忽略回傳值

Go 的一個強大特性是函式可以回傳多個值。這在錯誤處理、資料驗證和複雜計算中非常有用。

### 多個回傳值的特性
- **語法簡潔**: 直接在括號中列出回傳型態
- **常見模式**: `(結果, error)` 是 Go 的標準錯誤處理模式
- **解構賦值**: 使用多重賦值接收多個回傳值
- **全部接收**: 必須接收所有回傳值或明確忽略

### 忽略回傳值
- **空白識別符**: 使用 `_` 忽略不需要的回傳值
- **錯誤處理**: 建議不要忽略 error 回傳值

In [ ]:
package main

import (
    "fmt"
    "errors"
    "strconv"
)

// 基本的多回傳值：除法運算
func safeDivide(a, b float64) (float64, error) {
    if b == 0 {
        return 0, errors.New("division by zero")
    }
    return a / b, nil
}

// 查找函式：值和是否找到
func findInSlice(slice []int, target int) (int, bool) {
    for i, value := range slice {
        if value == target {
            return i, true
        }
    }
    return -1, false
}

func main() {
    // 接收所有回傳值
    result1, err1 := safeDivide(10, 2)
    if err1 != nil {
        fmt.Printf("錯誤: %v\n", err1)
    } else {
        fmt.Printf("10 / 2 = %.2f\n", result1)
    }
    
    // 忽略錯誤（不建議！）
    result2, _ := safeDivide(20, 4)
    fmt.Printf("20 / 4 = %.2f (忽略錯誤)\n", result2)
    
    // 查找操作
    numbers := []int{10, 20, 30, 40, 50}
    index, found := findInSlice(numbers, 30)
    if found {
        fmt.Printf("找到 30 在索引 %d\n", index)
    }
    
    // 實際應用：字串轉數字
    if num, err := strconv.Atoi("123"); err != nil {
        fmt.Printf("轉換錯誤: %v\n", err)
    } else {
        fmt.Printf("轉換成功: %d\n", num)
    }
}

## 5. 函式是值、函式型態宣告與匿名函式

介紹三個概念
- func是一等公民
- type function的特性與好處
- 匿名函式

### 函式是一等公民（first-class citizens）

在程式語言理論中，first-class citizen 指的是某個「語言元素」擁有和其他值一樣的地位，可以做以下事情：

- 被賦值給變數
- 作為參數傳遞
- 作為回傳值
- 可以存放在資料結構中（slice、map 等）

如果一種元素可以做到以上四點，就被稱為 first-class citizen



### 函式型態宣告(type function)
- **基本語法**: `type 函式型態名稱 func(參數列表) 回傳型態`

#### 用途

- **統一函式簽名**

例如，你有多個函式實作相同邏輯結構，可以用同一個型別作為參數或回傳值。


```go
func apply(a, b int, op Operation) int {
    return op(a, b)
}

func add(a, b int) int { return a + b }
func multiply(a, b int) int { return a * b }

result := apply(2, 3, add)      // 5
result2 := apply(2, 3, multiply) // 6

```

- **方便傳遞和組合函式**

可以把函式當作物件，存進 slice、map，或作為高階函式的參數。

- **強化可讀性**

Operation 比 func(int,int)int 更語意清楚，讓程式碼更 self-documenting。



In [ ]:
package main

import "fmt"

// 定義函式型態
type Operation func(int, int) int
type StringProcessor func(string) string

// 實作函式
func add(a, b int) int {
    return a + b
}

func multiply(a, b int) int {
    return a * b
}

// 高階函式：接收函式作為參數
func calculate(a, b int, op Operation) int {
    return op(a, b)
}

// 高階函式：回傳函式
func getOperation(name string) Operation {
    switch name {
    case "add":
        return add
    case "multiply":
        return multiply
    default:
        return func(a, b int) int { return 0 }
    }
}  

func main() {
    // 函式賦值給變數
    var op Operation = add
    fmt.Printf("使用函式變數: %d\n", op(3, 4))
    
    // 將函式作為參數
    result1 := calculate(5, 3, add)
    result2 := calculate(5, 3, multiply)
    fmt.Printf("add: %d, multiply: %d\n", result1, result2)
    
    // 匿名函式立即執行
    result3 := func(x, y int) int {
        return x*x + y*y
    }(3, 4)
    fmt.Printf("匿名函式結果: %d\n", result3)
    
    // 匿名函式賦值給變數
    square := func(x int) int {
        return x * x
    }
    fmt.Printf("square(5) = %d\n", square(5))
    
    // 從函式獲取函式
    addOp := getOperation("add")
    fmt.Printf("動態獲取的函式: %d\n", addOp(10, 20))
}

Cell[3]: Line 35 
 /var/folders/x6/lgk9k7t518v3kvp1t2q3r8kw0000gn/T/gonb_426a4017/main.go:36:1: expected declaration, found '{' (and 1 more errors)
 
 }
}

 {
 func outer() {
 // 在 outer 函式裡定義一個命名函式

ERROR: parsing go files in TempDir "/var/folders/x6/lgk9k7t518v3kvp1t2q3r8kw0000gn/T/gonb_426a4017": /var/folders/x6/lgk9k7t518v3kvp1t2q3r8kw0000gn/T/gonb_426a4017/main.go:36:1: expected declaration, found '{' (and 1 more errors)

## 6. 匿名函式 & Closure（閉包）

### 匿名函式
- **定義**: 沒有名稱的函式，可以直接定義和使用
- **語法**: `func(參數列表) 回傳型態 { 函式主體 }`
- **立即執行**: 可以定義後立即呼叫

function內部是不允許使用 named function, 只能使用匿名函式
```go

//not allowed
// func outer() {
//     // 在 outer 函式裡定義一個命名函式
//     func inner(a, b int) int {
//         return a + b
//     }

//     fmt.Println(inner(2,3))
// }

//這樣是允許的
func outer() {
    // 在 outer 函式裡定義一個命名函式
    inner := func(a, b int) int {
        return a + b
    }

    fmt.Println(inner(2,3))
}

```


### Runtime function

- **定義**: runtime才被組成的function, 基本上匿名函式就是runtime function


### Closure

Closure = runtime function + 捕捉外部變數 + (通常這個runtime function會被回傳出去)

Closure 是 Go 中一個強大的特性，它允許匿名函式存取和修改外部作用域的變數。closure 讓函式可以「記住」其定義時的環境。

### Closure 的特性
- **記憶環境**: 函式可以存取定義時的外部變數
- **變數共享**: 多個 closure 可以共享同一個外部變數
- **生命週期**: 外部變數的生命週期會延長到 closure 不再被引用為止
- **動態綁定**: closure 中的變數是動態綁定的，會反映最新值



In [ ]:
package main

import "fmt"

// 計數器工廠
func makeCounter() func() int {
    count := 0
    return func() int {
        count++
        return count
    }
}

// 加法器工廠
func makeAdder(x int) func(int) int {
    return func(y int) int {
        return x + y
    }
}

// 多個變數的 closure
func makeMultiplier(factor int) func(int) (int, string) {
    name := fmt.Sprintf("multiplier_%d", factor)
    return func(x int) (int, string) {
        return x * factor, name
    }
}

func main() {
    // 計數器示例
    counter1 := makeCounter()
    counter2 := makeCounter()
    
    fmt.Printf("counter1: %d\n", counter1()) // 1
    fmt.Printf("counter1: %d\n", counter1()) // 2
    fmt.Printf("counter2: %d\n", counter2()) // 1
    fmt.Printf("counter1: %d\n", counter1()) // 3
    
    // 加法器示例
    add10 := makeAdder(10)
    add100 := makeAdder(100)
    
    fmt.Printf("add10(5) = %d\n", add10(5))   // 15
    fmt.Printf("add100(5) = %d\n", add100(5)) // 105
    
    // 乘法器示例
    mul3 := makeMultiplier(3)
    result, name := mul3(7)
    fmt.Printf("%s: 7 * 3 = %d\n", name, result)
    
    // Closure 陷阱示例
    var funcs []func() int
    
    // 錯誤的方式; 只有在 1.22以前才是這樣， 目前實測1.24得到會拿到本來的value
    for i := 0; i < 3; i++ {
        funcs = append(funcs, func() int {
            return i // 注意：這裡會捕獲到最終的 i 值
        })
    }
    
    fmt.Println("錯誤的 closure:")
    for j, f := range funcs {
        fmt.Printf("funcs[%d]() = %d\n", j, f()) // 全部都是 3 (x) ; 才不會
    }
    //錯誤的 closure: 結果沒錯
    // funcs[0]() = 0
    // funcs[1]() = 1
    // funcs[2]() = 2
    
    // 正確的方式
    var correctFuncs []func() int
    for i := 0; i < 3; i++ {
        i := i // 創建新的變數; 1.22+之後不需要了; closure每次抓的i都會是新的變數
        correctFuncs = append(correctFuncs, func() int {
            return i
        })
    }
    
    fmt.Println("正確的 closure:")
    for j, f := range correctFuncs {
        fmt.Printf("correctFuncs[%d]() = %d\n", j, f()) // 0, 1, 2
    }
}

counter1: 1
counter1: 2
counter2: 1
counter1: 3
add10(5) = 15
add100(5) = 105
multiplier_3: 7 * 3 = 21
錯誤的 closure:
funcs[0]() = 0
funcs[1]() = 1
funcs[2]() = 2
正確的 closure:
correctFuncs[0]() = 0
correctFuncs[1]() = 1
correctFuncs[2]() = 2


## 7. defer 語句

`defer` 是 Go 中一個獨特的特性，用於延遲函式呼叫的執行，直到包含它的函式即將回傳時才執行。

**注意:** 會先defer執行完之後才會return

### defer 的特性
- **延遲執行**: 延遲到函式結束時執行
- **LIFO 順序**: 多個 defer 以後進先出（Last In, First Out）順序執行
- **參數即時求值**: defer 語句的參數在 defer 註冊時就會被求值
- **回傳值修改**: 可以修改具名回傳值

### 使用場景
- 資源清理（檔案關閉、連線關閉）
- 解鎖 mutex
- 復原操作
- 除錯和日誌記錄

### 兩種宣告方式

1. defer {expression}
如果有變數會先被catch住

2. defer func(){
    ...logic here
}()
如果有變數會再被執行的時候才抓進來

```go

    x := 1
    defer fmt.Printf("defer statement 中的 x: %d\n", x) // x 的值在這裡就被確定了

    defer func(){
        fmt.Println("defer function 中的 x: %d\n", x) // x 會在最後才會決定
    }()

    x = 2


```

In [6]:
package main

import (
    "fmt"
    "os"
)

// 模擬檔案操作
func readFile(filename string) error {
    fmt.Printf("開啟檔案: %s\n", filename)
    
    // 模擬開啟檔案
    //file := &os.File{} // 假設這是開啟的檔案
    
    // 使用 defer 確保檔案會被關閉
    defer func() {
        fmt.Printf("關閉檔案: %s\n", filename)
        // file.Close() // 實際情況下會關閉檔案
    }()
    
    fmt.Printf("讀取檔案: %s\n", filename)
    
    // 即使發生錯誤，defer 也會執行
    if filename == "error.txt" {
        return fmt.Errorf("讀取錯誤")
    }
    
    fmt.Printf("檔案讀取完成: %s\n", filename)
    return nil
}

// defer 的執行順序
func demonstrateOrder() {
    fmt.Println("開始函式")
    
    defer fmt.Println("第一個 defer")
    defer fmt.Println("第二個 defer")
    defer fmt.Println("第三個 defer")
    
    fmt.Println("函式主體")
    // 輸出順序：
    // 開始函式
    // 函式主體
    // 第三個 defer
    // 第二個 defer
    // 第一個 defer
}

// defer 修改回傳值
func modifyReturnValue() (result int) {
    defer func() {
        result += 10
        fmt.Printf("defer 修改回傳值為: %d\n", result)
    }()
    defer fmt.Printf("defer 修改回傳值為: %d\n", result+10)
    
    result = 5
    fmt.Printf("函式中的值: %d\n", result)
    return // 回傳 15 (5 + 10)
}

// defer 參數即時求值
func parameterEvaluation() {
    x := 1
    defer fmt.Printf("defer statement 中的 x: %d\n", x) // x 的值在這裡就被確定了

    defer func(){
        fmt.Println("defer function 中的 x: %d\n", x) // x 會在最後才會決定
    }()
    
    x = 2
    fmt.Printf("函式中的 x: %d\n", x)
    // 輸出:
    // 函式中的 x: 2
    // defer function 中的 x: 2
    // defer 中的 x: 1
}

func main() {
    fmt.Println("=== 檔案操作示例 ===")
    readFile("data.txt")
    fmt.Println()
    
    fmt.Println("=== 錯誤情況 ===")
    if err := readFile("error.txt"); err != nil {
        fmt.Printf("錯誤: %v\n", err)
    }
    fmt.Println()
    
    fmt.Println("=== defer 執行順序 ===")
    demonstrateOrder()
    fmt.Println()
    
    fmt.Println("=== 修改回傳值 ===")
    result := modifyReturnValue()
    fmt.Printf("最終回傳值: %d\n", result)
    fmt.Println()
    
    fmt.Println("=== 參數求值時機 ===")
    parameterEvaluation()
}

=== 檔案操作示例 ===
開啟檔案: data.txt
讀取檔案: data.txt
檔案讀取完成: data.txt
關閉檔案: data.txt

=== 錯誤情況 ===
開啟檔案: error.txt
讀取檔案: error.txt
關閉檔案: error.txt
錯誤: 讀取錯誤

=== defer 執行順序 ===
開始函式
函式主體
第三個 defer
第二個 defer
第一個 defer

=== 修改回傳值 ===
函式中的值: 5
defer 修改回傳值為: 10
defer 修改回傳值為: 15
最終回傳值: 15

=== 參數求值時機 ===
函式中的 x: 2
defer function 中的 x: %d
 2
defer statement 中的 x: 1


## 8. Go 是以值呼叫的

理解 Go 的參數傳遞機制對於寫出正確的程式很重要。Go 使用「值呼叫」（call by value），這意味著函式接收到的是參數的複本。

### 值呼叫的特性
- **複本傳遞**: 基本型態（int, float, bool, string）傳遞的是值的複本
- **結構體複本**: struct, array 傳遞時會複製整個結構體 ( 包含 傳入func as 參數，還有在local傳給另一個變數都會copy )
- **slice/map/channel**: 這些是引用型態，傳遞的是引用的複本
- **指標**: 可以通過指標間接修改原始值

### 重要概念
- 修改參數不會影響原始變數（除非是指標或引用型態）
- 大型結構體傳遞可能有性能影響
- 理解何時使用指標vs值傳遞

In [ ]:
package main

import "fmt"

// 值傳遞：基本型態
func modifyInt(x int) {
    x = 100
    fmt.Printf("函式內 x = %d\n", x)
}

// 指標傳遞：可以修改原始值
func modifyIntByPointer(x *int) {
    *x = 100
    fmt.Printf("函式內 *x = %d\n", *x)
}

// 結構體值傳遞
type Person struct {
    Name string
    Age  int
}

func modifyPerson(p Person) {
    p.Name = "Modified"
    p.Age = 999
    fmt.Printf("函式內 person: %+v\n", p)
}

func modifyPersonByPointer(p *Person) {
    p.Name = "Modified"
    p.Age = 999
    fmt.Printf("函式內 person: %+v\n", *p)
}

// slice 的特殊情況
func modifySlice(s []int) {
    // 修改元素：會影響原始 slice
    if len(s) > 0 {
        s[0] = 999
    }
    
    // append：可能不會影響原始 slice
    s = append(s, 100)
    fmt.Printf("函式內 slice: %v\n", s)
}

func modifySliceByPointer(s *[]int) {
    // 修改元素
    if len(*s) > 0 {
        (*s)[0] = 999
    }
    
    // append：會影響原始 slice
    *s = append(*s, 100)
    fmt.Printf("函式內 slice: %v\n", *s)
}

// map 的情況
func modifyMap(m map[string]int) {
    m["new"] = 100
    m["a"] = 999
    fmt.Printf("函式內 map: %v\n", m)
}

func main() {
    
    // 呼叫前 x = 10
    // 函式內 x = 100
    // 呼叫後 x = 10
    fmt.Println("=== 基本型態值傳遞 ===")
    x := 10
    fmt.Printf("呼叫前 x = %d\n", x)
    modifyInt(x)
    fmt.Printf("呼叫後 x = %d\n\n", x) // 值不變

    // === 指標傳遞 ===
    // 呼叫前 y = 10
    // 函式內 *x = 100
    // 呼叫後 y = 100   
    fmt.Println("=== 指標傳遞 ===")
    y := 10
    fmt.Printf("呼叫前 y = %d\n", y)
    modifyIntByPointer(&y)
    fmt.Printf("呼叫後 y = %d\n\n", y) // 值改變
    


    fmt.Println("=== 結構體值傳遞 ===")
    person1 := Person{Name: "Alice", Age: 25}
    fmt.Printf("呼叫前 person: %+v\n", person1)
    modifyPerson(person1)

     //this will copy the entire person includes the properties too
    personAssigned := person1
    personAssigned.Name = "Hank"
    personAssigned.Age = 42
    fmt.Printf("呼叫後 personAssigned: %+v\n", personAssigned)

    fmt.Printf("呼叫後 person: %+v\n\n", person1) // 值不變
    
    fmt.Println("=== 結構體指標傳遞 ===")
    person2 := Person{Name: "Bob", Age: 30}
    fmt.Printf("呼叫前 person: %+v\n", person2)
    modifyPersonByPointer(&person2)
    fmt.Printf("呼叫後 person: %+v\n\n", person2) // 值改變

    
    
    fmt.Println("=== slice 修改 ===")
    slice1 := []int{1, 2, 3}
    fmt.Printf("呼叫前 slice: %v\n", slice1)
    modifySlice(slice1)
    fmt.Printf("呼叫後 slice: %v\n\n", slice1) // 第一個元素改變，但沒有新元素
    
    fmt.Println("=== slice 指標修改 ===")
    slice2 := []int{1, 2, 3}
    fmt.Printf("呼叫前 slice: %v\n", slice2)
    modifySliceByPointer(&slice2)
    fmt.Printf("呼叫後 slice: %v\n\n", slice2) // 完全改變
    
    fmt.Println("=== map 修改 ===")
    map1 := map[string]int{"a": 1, "b": 2}
    fmt.Printf("呼叫前 map: %v\n", map1)
    modifyMap(map1)
    fmt.Printf("呼叫後 map: %v\n", map1) // map 是引用型態，會改變
}

=== 基本型態值傳遞 ===
呼叫前 x = 10
函式內 x = 100
呼叫後 x = 10

=== 指標傳遞 ===
呼叫前 y = 10
函式內 *x = 100
呼叫後 y = 100

=== 結構體值傳遞 ===
呼叫前 person: {Name:Alice Age:25}
函式內 person: {Name:Modified Age:999}
呼叫後 personAssigned: {Name:Hank Age:42}
呼叫後 person: {Name:Alice Age:25}

=== 結構體指標傳遞 ===
呼叫前 person: {Name:Bob Age:30}
函式內 person: {Name:Modified Age:999}
呼叫後 person: {Name:Modified Age:999}

=== slice 修改 ===
呼叫前 slice: [1 2 3]
函式內 slice: [999 2 3 100]
呼叫後 slice: [999 2 3]

=== slice 指標修改 ===
呼叫前 slice: [1 2 3]
函式內 slice: [999 2 3 100]
呼叫後 slice: [999 2 3 100]

=== map 修改 ===
呼叫前 map: map[a:1 b:2]
函式內 map: map[a:999 b:2 new:100]
呼叫後 map: map[a:999 b:2 new:100]


## 總結

在第五章中，我們深入學習了 Go 的函式系統：

### 核心概念
1. **函式基礎**: 宣告、呼叫、參數、回傳值
2. **具名回傳值**: 提高可讀性，支援裸回傳
3. **可變參數**: 使用 `...` 語法處理不定數量的參數
4. **多回傳值**: Go 的特色，常用於錯誤處理
5. **函式是值**: 函式可以賦值、傳遞、存儲
6. **匿名函式**: 靈活的臨時函式定義
7. **Closure**: 捕獲外部變數的強大機制
8. **defer**: 延遲執行，用於資源清理
9. **值呼叫**: 理解參數傳遞機制

### 最佳實踐
- 善用具名回傳值提高程式碼可讀性
- 不要忽略 error 回傳值
- 合理使用 defer 進行資源管理
- 理解值傳遞 vs 指標傳遞的差異
- 善用 closure 實現優雅的程式設計模式
- 注意 closure 中變數捕獲的陷阱

### 下章預告
下一章將學習指標，這是理解 Go 記憶體管理和高效程式設計的關鍵。我們將探討指標的基礎概念、使用場景和最佳實踐。